In [2]:
# !pip install -r req.txt

In [133]:
import math

In [61]:
from nltk.tokenize import sent_tokenize
import pandas as pd

In [ ]:
pd.set_option()

In [124]:
pd.set_option("display.max_colwidth",0)

In [62]:
from sentence_transformers import SentenceTransformer, util

In [260]:
class ClusterData:
    def __init__(self,model,community_threshold):
        self.model = model
        self.comm_thresh = community_threshold
    
    def do_cluster(self,request_id,input_sents:pd.DataFrame):
        try:
            if input_sents.shape[0]>0:
                sents = input_sents.text.tolist()
                embeddings = self.model.encode(sents,batch_size=32,convert_to_tensor=True)
                cluster_ind = util.community_detection(embeddings,threshold=self.comm_thresh,
                                        min_community_size=1)
                input_sents["cluster"]=None
                for i, ind in enumerate(cluster_ind):
                    input_sents.loc[ind,"cluster"]= i
                input_sents.cluster.fillna(-1,inplace=True)
                return input_sents

            else:
                    return input_sents
        except Exception as e:
            print(f"clustering failed with err {e}")
            return input_sents
            
        

In [82]:
# sent_tokenize("how are you? i never met you before")

In [83]:
# # get cluster 
# select sentence 
# handle single cluster

# do till word limit are not maced

In [84]:
text = '''The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price.
    The deal, first reported by The Real Deal, was for $150 million, according to a source familiar with the deal.
    Mubadala, an Abu Dhabi investment fund, purchased 90% of the building for $800 million in 2008.
    Real estate firm Tishman Speyer had owned the other 10%.
    The buyer is RFR Holding, a New York real estate company.
    Officials with Tishman and RFR did not immediately respond to a request for comments.
    It's unclear when the deal will close.
    The building sold fairly quickly after being publicly placed on the market only two months ago.
    The sale was handled by CBRE Group.
    The incentive to sell the building at such a huge loss was due to the soaring rent the owners pay to Cooper Union, a New York college, for the land under the building.
    The rent is rising from $7.75 million last year to $32.5 million this year to $41 million in 2028.
    Meantime, rents in the building itself are not rising nearly that fast.
    While the building is an iconic landmark in the New York skyline, it is competing against newer office towers with large floor-to-ceiling windows and all the modern amenities.
    Still the building is among the best known in the city, even to people who have never been to New York.
    It is famous for its triangle-shaped, vaulted windows worked into the stylized crown, along with its distinctive eagle gargoyles near the top.
    It has been featured prominently in many films, including Men in Black 3, Spider-Man, Armageddon, Two Weeks Notice and Independence Day.
    The previous sale took place just before the 2008 financial meltdown led to a plunge in real estate prices.
    Still there have been a number of high profile skyscrapers purchased for top dollar in recent years, including the Waldorf Astoria hotel, which Chinese firm Anbang Insurance purchased in 2016 for nearly $2 billion, and the Willis Tower in Chicago, which was formerly known as Sears Tower, once the world's tallest.
    Blackstone Group (BX) bought it for $1.3 billion 2015.
    The Chrysler Building was the headquarters of the American automaker until 1953, but it was named for and owned by Chrysler chief Walter Chrysler, not the company itself.
    Walter Chrysler had set out to build the tallest building in the world, a competition at that time with another Manhattan skyscraper under construction at 40 Wall Street at the south end of Manhattan. He kept secret the plans for the spire that would grace the top of the building, building it inside the structure and out of view of the public until 40 Wall Street was complete.
    Once the competitor could rise no higher, the spire of the Chrysler building was raised into view, giving it the title.
'''

In [65]:
sents = nltk.sent_tokenize(text)

In [232]:
sent_df = pd.DataFrame(sents,columns=["text"]).reset_index()

In [92]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [261]:
cluster_obj = ClusterData(model,0.4)

In [112]:
util.cos_sim(model.encode(sents[0]),model.encode(sents[9]))

tensor([[0.3535]])

In [288]:
# df = cluster_obj.do_cluster(1,sent_df)

In [216]:
# df.sort_values(by="cluster")

In [235]:
# sent_df

In [139]:
ratio = 0.25

In [140]:
filter_n = math.floor(ratio * sent_df.shape[0])

In [142]:
candidates = pd.DataFrame(columns=sent_df.columns)
# candidates_unique

In [151]:
cluster_count = sent_df.cluster.value_counts().to_frame().reset_index()

In [154]:
cluster_count.columns = ["cluster","count"]

In [158]:
sent_df = sent_df.merge(cluster_count)

In [282]:
# unique_cadidates = sent_df[sent_df['count']==1].reset_index(drop=True)

In [283]:
class SummExtract:
    def __init__(self,cluster_obj,model):
        self.cluster_obj = cluster_obj
        self.model = model

        
    def summerize(self,text,ratio=0.25):
        sents = nltk.sent_tokenize(text)
        sent_df = pd.DataFrame(sents,columns=["text"]).reset_index()
        
        filter_n = math.floor(ratio * sent_df.shape[0])
        sent_df = cluster_obj.do_cluster(1,sent_df)
        # print(sent_df)
        if "cluster" in sent_df.columns:
            candidates = pd.DataFrame(columns=sent_df.columns)
            cluster_count = sent_df.cluster.value_counts().to_frame().reset_index()
            cluster_count.columns = ["cluster","count"]
            sent_df = sent_df.merge(cluster_count)
    
            n = 0
            while candidates.shape[0] <filter_n:
                temp = sent_df[sent_df.cluster==n]
                if temp.shape[0]>1:
                    candidates = pd.concat([candidates,temp.iloc[:1]])
                n+=1

            centroids = candidates.text.tolist()
            unique_cadidates = sent_df[sent_df['count']==1].reset_index(drop=True)
            if candidates.shape[0]<filter_n:
                if len(centroids)>0:
                    sim_scores = util.cos_sim(self.model.encode(unique_cadidates.text.tolist()),model.encode(centroids))
                    order_to_select = sim_scores.mean(axis=1).argsort(descending=True)
                    ind = 0
                    while candidates.shape[0]<filter_n:
                        item = order_to_select[ind].item()
                        candidates = pd.concat([candidates,unique_cadidates[item:item+1]])
                        ind+=1
                else:

                    embeds = self.model.encode(unique_cadidates.text.to_list())
                    sim_scores = util.cos_sim(embeds,embeds)
                    for i in range(sim_scores.shape[0]):
                        sim_scores[i][i]=0
                    order_to_select = sim_scores.mean(axis=1).argsort(descending=True)
                    ind = 0
                    while candidates.shape[0]<filter_n:
                        item = order_to_select[ind].item()
                        candidates = pd.concat([candidates,unique_cadidates[item:item+1]])
                        ind+=1
            candidates = candidates.sort_values(by="index")

            return " ".join(candidates.text.tolist())

        else:
            print("failed")
            return ""
            
        
        
        
        

In [284]:
obj = SummExtract(cluster_obj,model)

In [286]:
obj.summerize(text)

"The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price. The deal, first reported by The Real Deal, was for $150 million, according to a source familiar with the deal. The incentive to sell the building at such a huge loss was due to the soaring rent the owners pay to Cooper Union, a New York college, for the land under the building. Meantime, rents in the building itself are not rising nearly that fast. Still there have been a number of high profile skyscrapers purchased for top dollar in recent years, including the Waldorf Astoria hotel, which Chinese firm Anbang Insurance purchased in 2016 for nearly $2 billion, and the Willis Tower in Chicago, which was formerly known as Sears Tower, once the world's tallest."

In [287]:
text

"The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price.\n    The deal, first reported by The Real Deal, was for $150 million, according to a source familiar with the deal.\n    Mubadala, an Abu Dhabi investment fund, purchased 90% of the building for $800 million in 2008.\n    Real estate firm Tishman Speyer had owned the other 10%.\n    The buyer is RFR Holding, a New York real estate company.\n    Officials with Tishman and RFR did not immediately respond to a request for comments.\n    It's unclear when the deal will close.\n    The building sold fairly quickly after being publicly placed on the market only two months ago.\n    The sale was handled by CBRE Group.\n    The incentive to sell the building at such a huge loss was due to the soaring rent the owners pay to Cooper Union, a New York college, for the land under the building.\n    The rent is rising from $7.75 million last year to $32.5 million this year 